In [2]:
import pandas as pd
import numpy as np
import hvplot.pandas
import holoviews as hv
import seaborn as sns

In [3]:
global density_plot
global mc_plot
global final_plot

In [4]:
# Load the provided CSV file to examine its contents
%run HED.ipynb
hed_df.head()

Stored 'hed_df' (DataFrame)


,Close
Date,
2022-01-02,1.1341
2022-01-03,1.1301
2022-01-04,1.1291
2022-01-05,1.1315
2022-01-06,1.1303


In [5]:
# Add return column
hed_df['Returns'] = hed_df['Close'].pct_change()

In [6]:
# Drop nulls
hed_df= hed_df.dropna()
hed_df.head()


,Close,Returns
Date,,
2022-01-03,1.1301,-0.003527
2022-01-04,1.1291,-0.000885
2022-01-05,1.1315,0.002126
2022-01-06,1.1303,-0.001061
2022-01-07,1.1362,0.005220


In [7]:
# Set up Monte Carlo simulation parameters
num_simulations = 250  # Default number of simulations
forecast_period = 252  # Number of days to simulate

In [8]:
# Create an empty dataframe for the simulated values
simulation_results = pd.DataFrame()

In [9]:
# Perform the Monte Carlo simulations
for i in range(num_simulations):
    simulated_prices = [hed_df['Close'].iloc[-1]]
    for day in range(1, forecast_period + 1):
        simulated_prices.append(simulated_prices[-1] * (1 + np.random.normal(hed_df['Returns'].mean(), hed_df['Returns'].std())))
    simulation_results[i] = simulated_prices

C:\Users\Jozweh\AppData\Local\Temp\ipykernel_48848\1848751618.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  simulation_results[i] = simulated_prices


In [10]:
simulation_results.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,1.065600,1.065600,1.065600,1.065600,1.065600,1.065600,1.065600,1.065600,1.065600,1.065600,...,1.065600,1.065600,1.065600,1.065600,1.065600,1.065600,1.065600,1.065600,1.065600,1.065600
1,1.061792,1.065123,1.070008,1.067934,1.058317,1.062159,1.069117,1.067050,1.060968,1.068130,...,1.067925,1.059037,1.065036,1.067953,1.072818,1.065433,1.057221,1.064862,1.066261,1.061320
2,1.061699,1.063507,1.079212,1.074021,1.064594,1.063705,1.074681,1.068037,1.063460,1.073641,...,1.068564,1.058872,1.068256,1.063966,1.067048,1.063133,1.056475,1.066739,1.058280,1.067757
3,1.057639,1.063770,1.073362,1.073652,1.058596,1.068187,1.069239,1.069589,1.069805,1.072062,...,1.063896,1.042165,1.072649,1.062594,1.068117,1.062236,1.060764,1.067159,1.055871,1.073061
4,1.063816,1.058785,1.071571,1.068725,1.064831,1.071503,1.068240,1.067420,1.071702,1.070434,...,1.069618,1.039099,1.077704,1.057340,1.068622,1.059053,1.047966,1.069691,1.047334,1.071399


In [11]:
# Genereate Monte Carlo plot
mc_plot = simulation_results.hvplot(
    xlabel='Days',
    ylabel = 'Price',
    title= 'Monte Carlo Simulation'
    )
mc_plot

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [22]:
# Calculate specific percentiles for the final cumulative returns
# percentiles = [10, 50, 90]  # Specified percentiles
# percentile_values = np.percentile(final_prices, percentiles)

# Creating a dictionary to display the percentiles with their corresponding values
# percentile_results = {f"{p}th Percentile": v for p, v in zip(percentiles, percentile_values)}
# percentile_results

In [26]:
# Histogram of the final values
final_values = simulation_results.iloc[-1]  # Grab the final row
lower_bound = np.percentile(final_values, 2.5)
upper_bound = np.percentile(final_values, 97.5)
final_values_plot=final_values.hvplot.hist(
    title='Distribution of Final Forecasted Exchange Rates',
    xlabel='Exchange Rate',
    ylabel='Frequency'
)
min_line=hv.VLine(lower_bound)
max_line=hv.VLine(upper_bound)

final_plot= final_values_plot * min_line * max_line 
final_plot


:Overlay
   .Histogram.I :Histogram   [252]   (252_count)
   .VLine.I     :VLine   [x,y]
   .VLine.II    :VLine   [x,y]

In [30]:
# Generate density plot of the final forecast 
density_plot=final_values.hvplot.kde(
    title='Density Plot of Final Forecasted Exchange Rates',
    xlabel='Exchange Rate',
    ylabel='Density'
)

density_plot


:Distribution   [252]   (Density)